In [33]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp

import os
import sys

from tqdm import tqdm

import re

from PIL import Image
import requests

import warnings
# warnings.filterwarnings('ignore')

In [2]:
data_path = "/opt/ml/data/"

In [3]:
users = pd.read_csv(data_path + 'users.csv')
books = pd.read_csv(data_path + 'books.csv')
train = pd.read_csv(data_path + 'train_ratings.csv')
test = pd.read_csv(data_path + 'test_ratings.csv')

In [4]:
print('users shape: ', users.shape)
print('books shape: ', books.shape)
print('ratings shape: ', train.shape)

users shape:  (68092, 3)
books shape:  (149570, 10)
ratings shape:  (306795, 3)


In [26]:
users.head()

,user_id,location,age
0,8,"timmins, ontario, canada",NaN
1,11400,"ottawa, ontario, canada",49.0
2,11676,"n/a, n/a, n/a",NaN
3,67544,"toronto, ontario, canada",30.0
4,85526,"victoria, british columbia, canada",36.0


In [11]:
train.head()

,user_id,isbn,rating
0,8,0002005018,4
1,67544,0002005018,7
2,123629,0002005018,8
3,200273,0002005018,8
4,210926,0002005018,9


In [10]:
train_df.head()

,user_id,isbn,rating,location,age,category,publisher,language,book_author,year_of_publication
0,8,0002005018,4,"timmins, ontario, canada",NaN,['Actresses'],HarperFlamingo Canada,en,Richard Bruce Wright,2001.0
1,67544,0002005018,7,"toronto, ontario, canada",30.0,['Actresses'],HarperFlamingo Canada,en,Richard Bruce Wright,2001.0
2,123629,0002005018,8,"kingston, ontario, canada",NaN,['Actresses'],HarperFlamingo Canada,en,Richard Bruce Wright,2001.0
3,200273,0002005018,8,"comber, ontario, canada",NaN,['Actresses'],HarperFlamingo Canada,en,Richard Bruce Wright,2001.0
4,210926,0002005018,9,"guelph, ontario, canada",NaN,['Actresses'],HarperFlamingo Canada,en,Richard Bruce Wright,2001.0


In [49]:
train_df_means = train_df.groupby('user_id')['rating'].mean()
train_df_total_mean = train_df['rating'].mean()


7.069714304340032

In [55]:
train_df_vars = train_df.groupby('user_id')['rating'].var()
train_df_total_var = train_df['rating'].var()

In [57]:
train_df_medians = train_df.groupby('user_id')['rating'].median()
train_df_total_median = train_df['rating'].median()

In [59]:
users['rating_avg'] = users['user_id'].apply(lambda x: train_df_means[x] if x in train_df_means else train_df_total_mean)
users['rating_var'] = users['user_id'].apply(lambda x: train_df_vars[x] if x in train_df_vars else train_df_total_var).fillna(0)
users['rating_median'] = users['user_id'].apply(lambda x: train_df_medians[x] if x in train_df_medians else train_df_total_median)
users

,user_id,location,age,rating_avg,rating_var,rating_median
0,8,"timmins, ontario, canada",NaN,4.428571,3.952381,4.0
1,11400,"ottawa, ontario, canada",49.0,6.750000,4.568182,7.0
2,11676,"n/a, n/a, n/a",NaN,6.779891,6.804415,7.0
3,67544,"toronto, ontario, canada",30.0,7.285714,0.238095,7.0
4,85526,"victoria, british columbia, canada",36.0,7.666667,2.627451,8.0
...,...,...,...,...,...,...
68087,278376,"danville, pennsylvania, usa",54.0,7.000000,0.000000,7.0
68088,278621,"victoria, delaware, canada",74.0,8.000000,0.000000,8.0
68089,278636,"irvington, alabama, usa",NaN,2.000000,0.000000,2.0
68090,278659,"vancouver, washington, usa",33.0,10.000000,0.000000,10.0


In [7]:
ratings1 = train.copy() ; ratings2 = test.copy()
ratings = pd.concat([ratings1, ratings2]).reset_index(drop=True)

In [9]:
context_df = ratings.merge(users, on='user_id', how='left').merge(books[['isbn', 'category', 'publisher', 'language', 'book_author', 'year_of_publication']], on='isbn', how='left')
train_df = ratings1.merge(users, on='user_id', how='left').merge(books[['isbn', 'category', 'publisher', 'language', 'book_author', 'year_of_publication']], on='isbn', how='left')
test_df = ratings2.merge(users, on='user_id', how='left').merge(books[['isbn', 'category', 'publisher', 'language', 'book_author', 'year_of_publication']], on='isbn', how='left')
